# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

    1. What is the appropriate test for this problem? Does CLT apply?

The appropriate test for this problem is a Chi-square test. This is because we are trying to find out if proportion of call-back is independent with white/black-sounding name. With Chi-square test, CLT does not apply.

    2. What are the null and alternative hypotheses?

Null hypothesis: white/black-sounding name does not have an impact on the chances of a call-back.

Alternative hypothesis: white/black-sounding name does have an impact on the chances of a call-back.

Establish that confidence level is 95%.

In [6]:
table = pd.crosstab(data.race, data.call)
table

call,0.0,1.0
race,,
b,2278,157
w,2200,235


In [7]:
chi2, p, dof, exp = stats.chi2_contingency(table.values)
print("chi2: {:.2f}, p-value: {:.5f}, dof: {}, exp: {}".format(chi2, p, dof, exp))

chi2: 16.45, p-value: 0.00005, dof: 1, exp: [[ 2239.   196.]
 [ 2239.   196.]]


Since p-value is much less than 5% (p-value = 0.005%), the null hypothesis should be rejected.

    3. Compute margin of error, confidence interval, and p-value.

In [8]:
# Computing the empirical proportions and empirical differences
total = table.loc['b', 0] + table.loc['b', 1]
empirical_prop_b = table.loc['b', 1]/total
empirical_prop_w = table.loc['w', 1]/total
empirical_diff = empirical_prop_w - empirical_prop_b
print(" empirical difference in proportions is: {:.3f}".format(empirical_diff))

 empirical difference in proportions is: 0.032


In [9]:
# Generating samples of proportions for each using binomial distribution for modelling
sample_prop_b = np.random.binomial(total, empirical_prop_b, 1000000)/total
sample_prop_w = np.random.binomial(total, empirical_prop_w, 1000000)/total

In [10]:
# Obtaining samples of differences in proportions
sample_diff = sample_prop_w - sample_prop_b

In [11]:
# As per null hypothesis, mean is shifted to 0
sample_diff_shifted = sample_diff - np.mean(sample_diff)

In [12]:
# Working out z-statistic for the empirical difference
z_stat = (empirical_diff - 0)/np.std(sample_diff_shifted)
print("z-statistic: {:.2f}".format(z_stat))

z-statistic: 4.12


In [13]:
# Working out z-critical for 95% (97.5% for two-tails)
z_crit = stats.norm.ppf(0.975)
print("z-critical: {:.2f}".format(z_crit))

z-critical: 1.96


In [14]:
# Working out p-value
p_value = np.sum(sample_diff_shifted > empirical_diff)/len(sample_diff_shifted)
print("p-value: {:5f}".format(p_value))

p-value: 0.000011


In [15]:
# Working out confidence level
confidence_interval = np.percentile(sample_diff_shifted, [2.5, 97.5])
print("confidence level is: {}".format(confidence_interval))

confidence level is: [-0.01519847  0.01519168]


In [16]:
# Working out margin of error
margin_of_error = abs(confidence_interval) - 0
print("margin of error is: {:.3f}".format(np.mean(margin_of_error)))

margin of error is: 0.015


    4. Write a story describing the statistical significance in the context of the original problem.

The Chi-square test done above concluded that the call-back proportion isn't independent of race. In order to re-confirm the finding, logistic regression has been used on the available data. Using the regression model, the strongest predictors of call-back are largely firstnames of the candidates, which are reflections of the race. Therefore, based on the available data and associated analyses, it can be concluded that call-back proportion is impacted by the candidate's race. However, through the analysis, it was found that honours and special skills are both potential predictors of call-back.

In [17]:
# Importing relevant packages for logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split


/Users/AlastairWong/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
# Initial view of data
print(data.shape)
print(list(data.columns))

(4870, 65)
['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer', 'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool', 'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race', 'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite', 'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq', 'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales', 'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip', 'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip', 'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport', 'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq', 'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade', 'busservice', 'othservice', 'missind', 'ownership']


In [19]:
# Initial view of data
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [20]:
# Initialising a list of potential columns to keep and later on eliminating columns which are not used.
to_keep = list(data.columns)

In [21]:
# Viewing columns along with their associated data type
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [22]:
# Exploring each column to determine whether column should be used. If the meaning of columns are not 
# understood, columns are to be removed. Categorical columns are to be converted to 'category's.
data.id.unique()

array(['b', '281', '221', '366', '349', '254', '1', '393', '230', '290',
       '402', '185', '358', '375', 'a', '359', '11', '403', '264', '291',
       '315', '81', '187', '404', '292', '265', '360', '377', '232', '316',
       '293', '233', '378', '317', '266', '405', '188', '361', '379',
       '318', '362', '14', '294', '267', '406', '234', '189', '84', '295',
       '380', '268', '319', '407', '363', '408', '381', '86', '269', '191',
       '364', '296', '237', '87', '365', '142', '297', '270', '192', '321',
       '271', '298', '238', '383', '322', '193', '19', '411', '239', '299',
       '194', '144', '272', '222', '282', '255', '367', '394', '306',
       '324', '240', '412', '90', '300', '196', '301', '274', '325', '413',
       '386', '242', '414', '326', '275', '22', '387', '197', '147', '92',
       '302', '415', '93', '303', '388', '276', '148', '243', '198', '304',
       '24', '244', '416', '149', '277', '389', '245', '200', '95', '278',
       '417', '25', '329', '150'

In [23]:
data.ad.unique()

array(['1', '10', '100', '100b', '101b', '102', '103', '103b', '104',
       '104b', '105', '105b', '107', '107b', '108', '108b', '109', '109b',
       '10b', '11', '110b', '111', '111b', '112b', '117', '118', '119',
       '11b', '12', '121', '122', '127', '12b', '13', '132', '133', '134',
       '137', '138', '13b', '14', '141', '144', '14b', '15', '152', '153',
       '154', '156', '157', '158', '159', '15b', '16', '161', '162', '163',
       '164', '165', '166', '168', '169', '16b', '17', '170', '171', '172',
       '173', '174', '175', '176', '177', '178', '179', '17b', '18', '180',
       '181', '182', '183', '184', '185', '186', '187', '189', '18b', '19',
       '190', '191', '192', '193', '194', '196', '197', '198', '199', '1b',
       '2', '20', '201', '202', '203', '20b', '21', '211', '214', '215',
       '217', '218', '219', '21b', '22', '220', '221', '222', '223', '224',
       '225', '226', '227', '228', '229', '22b', '23', '232', '233', '234',
       '235', '238', '239', 

In [24]:
to_keep.remove('id')
to_keep.remove('ad')

In [25]:
data.education.unique()

array([4, 3, 1, 2, 0])

In [26]:
data.education = data.education.astype('category')

In [27]:
data.ofjobs = data.ofjobs.astype('category')


In [28]:
data.yearsexp.unique()

array([ 6, 22,  5, 21,  3,  8,  4,  2,  7,  9, 13, 19, 12, 11, 10, 23,  1,
       14, 18, 26, 15, 25, 16, 20, 17, 44])

In [29]:
data.honors.unique()

array([0, 1])

In [30]:
data.honors = data.honors.astype('category')

In [31]:
data.volunteer.unique()

array([0, 1])

In [32]:
data.volunteer = data.volunteer.astype('category')

In [33]:
data.military.unique()

array([0, 1])

In [34]:
data.military = data.military.astype('category')

In [35]:
data.empholes.unique()

array([1, 0])

In [36]:
data.empholes = data.empholes.astype('category')

In [37]:
to_keep.remove('occupspecific')

In [38]:
to_keep.remove('occupbroad')

In [39]:
data.workinschool.unique()

array([0, 1])

In [40]:
data.workinschool = data.workinschool.astype('category')

In [41]:
data.email = data.email.astype('category')

In [42]:
data.computerskills = data.computerskills.astype('category')

In [43]:
data.specialskills = data.specialskills.astype('category')

In [44]:
data.sex = data.sex.astype('category')

In [45]:
# The race is to be removed as it has association with firstname
to_keep.remove('race')

In [46]:
to_keep.remove('h')

In [47]:
to_keep.remove('l')

In [48]:
# Remaining columns are to be removed for the purpose of this exercise.
del to_keep[14:]

In [49]:
data_refined = data[to_keep]

In [50]:
# Columns to be used for the logistic regression
data_refined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 14 columns):
education         4870 non-null category
ofjobs            4870 non-null category
yearsexp          4870 non-null int8
honors            4870 non-null category
volunteer         4870 non-null category
military          4870 non-null category
empholes          4870 non-null category
workinschool      4870 non-null category
email             4870 non-null category
computerskills    4870 non-null category
specialskills     4870 non-null category
firstname         4870 non-null object
sex               4870 non-null category
call              4870 non-null float32
dtypes: category(11), float32(1), int8(1), object(1)
memory usage: 153.6+ KB


In [51]:
# Creating dummy variables for categorical variables
cat_vars = ['education', 'ofjobs', 'honors', 'volunteer', 'military', 'empholes', 'workinschool', 'email', \
            'computerskills', 'specialskills', 'firstname', 'sex']
for var in cat_vars:
    cat_list = pd.get_dummies(data_refined[var], prefix=var)
    data_refined = data_refined.join(cat_list)

In [52]:
# Generating a comprehensive list of variables to be used including dummy variables
data_vars = data_refined.columns.values.tolist()
final_keep = [i for i in data_vars if i not in cat_vars]

In [53]:
# Separating dependent and independent variables
y = 'call'
x = [i for i in final_keep if i != 'call']

In [54]:
# Using Recursive Feature Elimination (RFE) to choose the best features
from sklearn.feature_selection import RFE
logreg = LogisticRegression()
rfe = RFE(logreg, 10).fit(data_refined[x], data_refined[y])
# print(rfe.support_)
# print(rfe.ranking_)
ranks = sorted(zip(rfe.ranking_, x))
ranks

[(1, 'firstname_Aisha'),
 (1, 'firstname_Brad'),
 (1, 'firstname_Keisha'),
 (1, 'firstname_Lakisha'),
 (1, 'firstname_Rasheed'),
 (1, 'firstname_Tamika'),
 (1, 'firstname_Tanisha'),
 (1, 'firstname_Tremayne'),
 (1, 'honors_0'),
 (1, 'specialskills_0'),
 (2, 'firstname_Kristen'),
 (3, 'firstname_Carrie'),
 (4, 'firstname_Jay'),
 (5, 'firstname_Darnell'),
 (6, 'education_0'),
 (7, 'ofjobs_1'),
 (8, 'empholes_0'),
 (9, 'firstname_Tyrone'),
 (10, 'firstname_Kareem'),
 (11, 'ofjobs_3'),
 (12, 'computerskills_1'),
 (13, 'firstname_Todd'),
 (14, 'firstname_Jermaine'),
 (15, 'volunteer_0'),
 (16, 'volunteer_1'),
 (17, 'email_0'),
 (18, 'firstname_Leroy'),
 (19, 'military_1'),
 (20, 'firstname_Meredith'),
 (21, 'firstname_Laurie'),
 (22, 'ofjobs_5'),
 (23, 'firstname_Matthew'),
 (24, 'firstname_Greg'),
 (25, 'sex_m'),
 (26, 'sex_f'),
 (27, 'firstname_Hakim'),
 (28, 'firstname_Ebony'),
 (29, 'firstname_Sarah'),
 (30, 'education_3'),
 (31, 'workinschool_0'),
 (32, 'workinschool_1'),
 (33, 'specia

In [55]:
# Creating a new list with the best features selected by RFE
selected_keep = []
for i, j in ranks:
    if i == 1:
        selected_keep.append(j)
selected_keep

['firstname_Aisha',
 'firstname_Brad',
 'firstname_Keisha',
 'firstname_Lakisha',
 'firstname_Rasheed',
 'firstname_Tamika',
 'firstname_Tanisha',
 'firstname_Tremayne',
 'honors_0',
 'specialskills_0']

In [56]:
# Workaround to debug chisqprob error
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [57]:
# Using stats model to confirm that most of the p-values are below 0.05
import statsmodels.api as sm
logit_model = sm.Logit(data_refined[y], data_refined[selected_keep])
stat_result = logit_model.fit()
print(stat_result.summary())

/Users/AlastairWong/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Optimization terminated successfully.
         Current function value: 0.274281
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                   call   No. Observations:                 4870
Model:                          Logit   Df Residuals:                     4860
Method:                           MLE   Df Model:                            9
Date:                Thu, 30 Nov 2017   Pseudo R-squ.:                 0.02033
Time:                        20:04:10   Log-Likelihood:                -1335.7
converged:                       True   LL-Null:                       -1363.5
                                        LLR p-value:                 1.011e-08
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
firstname_Aisha       -1.4817      0.511     -2.900      0.004      -2.483      -0.480
first

In [58]:
# Using train_test_split to split data into training and testing data
x_train, x_test, y_train, y_test = train_test_split(data_refined[selected_keep], \
                                                    data_refined[y], test_size=0.3, random_state=42)
# Fitting line with logistic regression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [59]:
# Using logistic regression to predict values using x-test and printing accuracy
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.93


    5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?


According to my analysis, race/name is one of the most important factor in callback success, as the best predictors are largely constituted of names. However, it was also found that honors and special skills are also predictors which are potentially stronger predictors compared to race/name. I would amend my analysis by expanding the scope to also look at other columns in the table in my logistic regression (note that a large fraction of the columns had been left out for the purpose of this exercise).